# PoC obtener links partidos

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


ModuleNotFoundError: No module named 'selenium'

In [ ]:
# Configuración de Selenium
options = Options()
# Descomenta la siguiente línea si quieres que el navegador se ejecute en modo headless (sin interfaz gráfica)
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL con la tabla de partidos
url_base = "https://www.laliganacional.com.ar/laliga/page/estadisticas#e1"
driver.get(url_base)

# Esperar a que la tabla de partidos esté visible.
# Esto asegura que el contenido dinámico de la página se ha cargado.
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table.table-striped tbody tr"))
    )
    print("Tabla de partidos encontrada y cargada.")
except Exception as e:
    print(f"Error: No se pudo encontrar la tabla de partidos. {e}")
    driver.quit()
    exit()

# Obtener el código fuente de la página completa después de la carga dinámica
page_source = driver.page_source
driver.quit()

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Buscar todos los enlaces a las estadísticas de los partidos.
# Se busca la etiqueta <a> cuyo atributo href contenga la URL base de los partidos individuales.
match_links = []
# La URL de un partido tiene el formato https://estadisticascabb.gesdeportiva.es/partido/...
for link_tag in soup.find_all("a", href=lambda href: href and "estadisticascabb.gesdeportiva.es/partido/" in href):
    full_link = link_tag.get('href')
    # Añadir el enlace a la lista si aún no está (para evitar duplicados)
    if full_link not in match_links:
        match_links.append(full_link)

# Imprimir el total de enlaces encontrados
print(f"Se encontraron {len(match_links)} enlaces de partidos.")

# Imprimir los primeros 10 enlaces para verificar
print("\nPrimeros 10 enlaces de partidos:")
for link in match_links[:10]:
    print(link)

# Ahora tienes la lista `match_links` con todas las URLs de los partidos.
# Puedes usar esta lista para iterar y aplicar tu código de scraping del mapa de tiro.